<a href="https://colab.research.google.com/github/prabhu2599/VectorBehta-Projects/blob/main/Split_Floders_Auto_Kaggle_direct_with_PeachySpecials.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install split-folders

In [ ]:
Data = '../input/sarscov2-ctscan-dataset'

In [ ]:
import splitfolders
splitfolders.ratio(Data, output="output", seed=1337, ratio=(.8, 0.1,0.1)) 

Copying files: 2481 files [00:19, 124.55 files/s]


In [ ]:
import sklearn
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss

In [ ]:
import os
from glob import glob
import matplotlib.pyplot as plt
import random
import cv2
import pandas as pd
import numpy as np
import matplotlib.gridspec as gridspec
import seaborn as sns
import zlib
import itertools
import sklearn
import itertools
import scipy
import skimage
from skimage.transform import resize
import csv
from tqdm import tqdm
from sklearn import model_selection
from sklearn.model_selection import train_test_split, learning_curve,KFold,cross_val_score,StratifiedKFold
from sklearn.utils import class_weight
from sklearn.metrics import confusion_matrix
import keras
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, Lambda, MaxPool2D, BatchNormalization
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras import models, layers, optimizers
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.utils import class_weight
from keras.optimizers import SGD, RMSprop, Adam, Adagrad, Adadelta, RMSprop
from keras.models import Sequential, model_from_json
from keras.layers import Activation,Dense, Dropout, Flatten, Conv2D, MaxPool2D,MaxPooling2D,AveragePooling2D, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras import backend as K
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.applications.inception_v3 import InceptionV3
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
#from keras.applications.mobilenet import MobileNet
#from sklearn.metrics import roc_auc_score
#from sklearn.metrics import roc_curve
#from sklearn.metrics import auc
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [ ]:
!pip install efficientnet-pytorch

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=9513d9683aebd2fbd8843e5c2acbc4a7230ea13e1878cf536dbd74ca1a5f491a
  Stored in directory: /root/.cache/pip/wheels/0e/cc/b2/49e74588263573ff778da58cc99b9c6349b496636a7e165be6
Successfully built efficientnet-pytorch


In [ ]:
# Imports
import os
import torch
import torch.nn.functional as F
import numpy as np
from torch import nn, optim
from torch.utils.data import DataLoader
from tqdm import tqdm
from efficientnet_pytorch import EfficientNet
# from utils import check_accuracy, load_checkpoint, save_checkpoint

In [ ]:
def check_accuracy(
    loader, model, loss_fn, input_shape=None, toggle_eval=True, print_accuracy=True
):
    """
    Check accuracy of model on data from loader
    """
    if toggle_eval:
        model.eval()
    device = next(model.parameters()).device
    num_correct = 0
    num_samples = 0

    y_preds = []
    y_true = []

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            if input_shape:
                x = x.reshape(x.shape[0], *input_shape)
            scores = model(x)
            predictions = torch.sigmoid(scores) > 0.5
            y_preds.append(torch.clip(torch.sigmoid(scores), 0.005, 0.995).cpu().numpy())
            y_true.append(y.cpu().numpy())
            num_correct += (predictions.squeeze(1) == y).sum()
            num_samples += predictions.size(0)

    accuracy = num_correct / num_samples

    if toggle_eval:
        model.train()

    if print_accuracy:
        print(f"Accuracy: {accuracy * 100:.2f}%")
#         print(log_loss(np.concatenate(y_true, axis=0), np.concatenate(y_preds, axis=0)))

    return accuracy


def save_checkpoint(state, filename="../working/my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    torch.save(state, filename)


def load_checkpoint(checkpoint, model):
    print("=> Loading checkpoint")
    model.load_state_dict(checkpoint["state_dict"])


def create_submission(model, model_name, files_dir):
    my_transforms = {
        "base": A.Compose(
            [
                A.Resize(height=240, width=240),
                A.Normalize(
                    mean=[0.485, 0.456, 0.406],
                    std=[0.229, 0.224, 0.225],
                    max_pixel_value=255.0,
                ),
                ToTensorV2(),
            ]
        ),
        "horizontal_flip": A.Compose(
            [
                A.Resize(height=240, width=240),
                A.HorizontalFlip(p=1.0),
                A.Normalize(
                    mean=[0.485, 0.456, 0.406],
                    std=[0.229, 0.224, 0.225],
                    max_pixel_value=255.0,
                ),
                ToTensorV2(),
            ]
        ),
        "vertical_flip": A.Compose(
            [
                A.Resize(height=240, width=240),
                A.VerticalFlip(p=1.0),
                A.Normalize(
                    mean=[0.485, 0.456, 0.406],
                    std=[0.229, 0.224, 0.225],
                    max_pixel_value=255.0,
                ),
                ToTensorV2(),
            ]
        ),
        "coloring": A.Compose(
            [
                A.Resize(height=240, width=240),
                A.ColorJitter(p=1.0),
                A.Normalize(
                    mean=[0.485, 0.456, 0.406],
                    std=[0.229, 0.224, 0.225],
                    max_pixel_value=255.0,
                ),
                ToTensorV2(),
            ]
        ),
        "rotate": A.Compose(
            [
                A.Resize(height=240, width=240),
                A.Rotate(p=1.0, limit=45),
                A.Normalize(
                    mean=[0.485, 0.456, 0.406],
                    std=[0.229, 0.224, 0.225],
                    max_pixel_value=255.0,
                ),
                ToTensorV2(),
            ]
        ),
        "shear": A.Compose(
            [
                A.Resize(height=240, width=240),
                A.IAAAffine(p=1.0),
                A.Normalize(
                    mean=[0.485, 0.456, 0.406],
                    std=[0.229, 0.224, 0.225],
                    max_pixel_value=255.0,
                ),
                ToTensorV2(),
            ]
        ),
    }

    for t in ["base", "horizontal_flip", "vertical_flip", "coloring", "rotate", "shear"]:
        predictions = []
        labels = []
        all_files = []
        test_dataset = MyDataset(root=files_dir, transform=my_transforms[t])
        test_loader = DataLoader(
            test_dataset, batch_size=32, num_workers=4, shuffle=False, pin_memory=True
        )
        model.eval()

        for idx, (x, y, filenames) in enumerate(tqdm(test_loader)):
            x = x.to(config.DEVICE)
            with torch.no_grad():
                outputs = (
                    torch.clip(torch.sigmoid(model(x)), 0.005, 0.995).squeeze(1).cpu().numpy()
                )
                predictions.append(outputs)
                labels += y.numpy().tolist()
                all_files += filenames

        df = pd.DataFrame(
            {
                "id": np.arange(
                    1,
                    (len(predictions) - 1) * predictions[0].shape[0]
                    + predictions[-1].shape[0]
                    + 1,
                ),
                "label": np.concatenate(predictions, axis=0),
            }
        )
        df.to_csv(f"../working/predictions_test/submission_{model_name}_{t}.csv", index=False)

        model.train()
        print(f"Created submission file for model {model_name} and transform {t}")


def save_feature_vectors(model, loader, output_size=(1, 1), file="trainb7"):
    model.eval()
    images, labels = [], []

    for idx, (x, y) in enumerate(tqdm(loader)):
        x = x.to(DEVICE)

        with torch.no_grad():
            features = model.extract_features(x)
            features = F.adaptive_avg_pool2d(features, output_size=output_size)
        images.append(features.reshape(x.shape[0], -1).detach().cpu().numpy())
        labels.append(y.numpy())

    np.save(f"../working/X_{file}.npy", np.concatenate(images, axis=0))
    np.save(f"../working/y_{file}.npy", np.concatenate(labels, axis=0))
    model.train()

def train_one_epoch(loader, model, loss_fn, optimizer, scaler):
    loop = tqdm(loader)

    for batch_idx, (data, targets) in enumerate(loop):
        data = data.to(DEVICE)
        targets = targets.to(DEVICE).unsqueeze(1).float()

        with torch.cuda.amp.autocast():
            scores = model(data)
            loss = loss_fn(scores, targets)

        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        loop.set_postfix(loss=loss.item())

In [ ]:
from torchvision.datasets import ImageFolder


In [ ]:
TRAIN_DIR = "./output/train"
VAL_DIR = "./output/val"

In [ ]:
imagenet_stats = ([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
import torchvision.transforms as T

train_tfms = T.Compose([
    T.Resize((256,256)),
#    T.CenterCrop(256),
#    T.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
#    T.RandomCrop(32, padding=4, padding_mode='reflect'),
#     T.RandomHorizontalFlip(), 
#     T.RandomRotation(10),
    
#    T.Normalize(*imagenet_stats,inplace=True), 
#    T.RandomErasing(inplace=True),
    T.ToTensor(),
])

valid_tfms = T.Compose([
    T.Resize((256,256)),
#    T.CenterCrop(256),
    T.ToTensor(),
#    T.Normalize(*imagenet_stats)
])

In [ ]:
NUM_WORKERS = 8
BATCH_SIZE = 32
PIN_MEMORY = True
LOAD_MODEL = True
SAVE_MODEL = True
CHECKPOINT_FILE = "b7.pth.tar"
WEIGHT_DECAY = 1e-4
LEARNING_RATE = 1e-4
NUM_EPOCHS = 2
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
# DEVICE = "cpu"

In [ ]:
def call_this_firstuu():
    model = EfficientNet.from_pretrained("efficientnet-b7")
    model._fc = nn.Linear(2560, 1)
    train_dataset = ImageFolder ( TRAIN_DIR , transform=train_tfms)
    test_dataset = ImageFolder ( VAL_DIR , transform=train_tfms ) 
#     train_dataset = CatDog(root="../input/10-monkey-species/training/", transform=basic_transform)
#     test_dataset = CatDog(root="../input/10-monkey-species/validation/", transform=basic_transform)
    train_loader = DataLoader(
        train_dataset,
        shuffle=True,
        batch_size=BATCH_SIZE,
        num_workers=NUM_WORKERS,
        pin_memory=True,
    )
    test_loader = DataLoader(
        test_dataset,
        shuffle=False,
        batch_size=BATCH_SIZE,
        num_workers=NUM_WORKERS,
    )
    model = model.to(DEVICE)

    scaler = torch.cuda.amp.GradScaler()
    loss_fn = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(
        model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY
    )

    if LOAD_MODEL and CHECKPOINT_FILE in os.listdir():
        load_checkpoint(torch.load(CHECKPOINT_FILE), model)

    for epoch in range(NUM_EPOCHS):
        train_one_epoch(train_loader, model, loss_fn, optimizer, scaler)
        check_accuracy(train_loader, model, loss_fn)

    if SAVE_MODEL:
        checkpoint = {"state_dict": model.state_dict(), "optimizer": optimizer.state_dict()}
        save_checkpoint(checkpoint, filename=CHECKPOINT_FILE)

    save_feature_vectors(model, train_loader, output_size=(1, 1), file="train_b7")
    save_feature_vectors(model, test_loader, output_size=(1, 1), file="test_b7")


In [ ]:
import torch
torch.cuda.empty_cache()

import gc
gc.collect()

242

In [ ]:
def train_one_epoch(loader, model, loss_fn, optimizer, scaler):
    loop = tqdm(loader)

    for batch_idx, (data, targets) in enumerate(loop):
        data = data.to(DEVICE)
        targets = targets.to(DEVICE).unsqueeze(1).float()

        with torch.cuda.amp.autocast():
            scores = model(data)
            loss = loss_fn(scores, targets)

        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        loop.set_postfix(loss=loss.item())

In [ ]:
call_this_firstuu()

Loaded pretrained weights for efficientnet-b7
=> Loading checkpoint


  0%|          | 0/62 [00:00<?, ?it/s]

Accuracy: 92.99%


100%|██████████| 62/62 [01:11<00:00,  1.15s/it, loss=0.021]  


Accuracy: 95.97%
=> Saving checkpoint


100%|██████████| 8/8 [00:04<00:00,  1.93it/s]


In [ ]:
X = np.load(f'../working/X_train_b7.npy')
y = np.load(f'../working/y_train_b7.npy')

print(f"Training data shape: {X.shape}, labels shape: {y.shape}")

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=1337)

Training data shape: (1984, 2560), labels shape: (1984,)


In [ ]:
clf = LogisticRegression(max_iter=20000)
clf.fit(X_train, y_train)

LogisticRegression(max_iter=20000)

In [ ]:
# Check on validation
val_preds= clf.predict_proba(X_val)[:,1]
print(f"On validation set:")
print(f"Accuracy: {clf.score(X_val, y_val)}")
# print(f"LOG LOSS: {log_loss(y_val, val_preds)} ")
# print("%--------------------------------------------------%")

# # # Get predictions on test set
# # print("Getting predictions for test set")
# # X_test = np.load(f'../input/X_test_b7.npy')
# # X_test_preds = clf.predict_proba(X_test)[:,1]
# # df = pd.DataFrame({'id': np.arange(1, 12501), 'label': np.clip(X_test_preds, 0.005, 0.995)})
# # df.to_csv(f"submissions/mysubmission.csv", index=False)
# # print("Done getting predictions!")

On validation set:
Accuracy: 0.9849246231155779
